In [1]:
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import Sequential

import os 
from PIL import Image ,UnidentifiedImageError, ImageFile
from pathlib import Path


In [2]:
data_dir = Path("Data")

dataset_info = {}

for split in ["Train_Data", "Test_Data"]:
    split_path = data_dir / split
    dataset_info[split] = {}
    for class_name in ["_Fire", "Non_Fire"]:
        class_path = split_path / class_name
        image_files = list(class_path.glob("*.jpg"))  # Or *.png, *.jpeg, etc.
        count = len(image_files)
        dimensions = []
        for image_file in image_files[:5]:
            try:
                with Image.open(image_file) as img:
                    dimensions.append(img.size + (len(img.getbands()),))  # (width, height, channels)
            except Exception as e:
                print(f"Error opening image {image_file}: {e}")
        dataset_info[split][class_name] = {"count": count, "dimensions": dimensions}


for split, classes in dataset_info.items():
    print(f"\n{split.upper()} DATASET")
    for class_name, info in classes.items():
        count = info.get('count', 0)  # Use .get() with default value
        dimensions = info.get('dimensions', []) #Use .get() with default value
        print(f" Classe '{class_name}': {count} images")
        if dimensions: # check if dimension list is not empty
            print(f" Dimensions (exemples): {dimensions[:5]}") # print only the first 5 dimensions
        else:
            print("No dimensions available")



TRAIN_DATA DATASET
 Classe '_Fire': 2111 images
 Dimensions (exemples): [(250, 250, 3), (250, 250, 3), (250, 250, 3), (250, 250, 3), (194, 260, 3)]
 Classe 'Non_Fire': 2500 images
 Dimensions (exemples): [(250, 250, 3), (250, 250, 3), (250, 250, 3), (250, 250, 3), (600, 403, 3)]

TEST_DATA DATASET
 Classe '_Fire': 25 images
 Dimensions (exemples): [(1200, 749, 3), (1200, 900, 3), (760, 368, 3), (1440, 810, 3), (286, 176, 3)]
 Classe 'Non_Fire': 25 images
 Dimensions (exemples): [(800, 600, 3), (1152, 1536, 3), (930, 558, 3), (545, 482, 4), (700, 525, 3)]


In [3]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

def clean_image_dataset(data_dir):
    """
    Cleans a dataset of images by removing corrupted or unreadable files.

    Args:
        data_dir: The root directory of the dataset (containing subdirectories for classes).
    """
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            print(f"Checking {class_name} directory...")
            for filename in os.listdir(class_dir):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')): # Add other extensions if needed
                    image_path = os.path.join(class_dir, filename)
                    try:
                        img = Image.open(image_path)
                        img.verify()  # Verify image integrity
                        img.close() # Close the file after verification
                    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
                        print(f"Corrupted or unreadable image found: {image_path}. Removing...")
                        try:
                            os.remove(image_path)
                            print(f"Successfully removed: {image_path}")
                        except OSError as e:
                            print(f"Error removing file: {e}")
                    except Exception as e:
                        print(f"Other error: {e} with image {image_path}")
                        
# Example usage:
train_data_dir = "Data/Train_Data" # Replace with your training data directory
test_data_dir = "Data/Test_Data" # Replace with your test data directory

clean_image_dataset(train_data_dir)
clean_image_dataset(test_data_dir)

print("Cleaning process finished.")


Checking Non_Fire directory...
Checking _Fire directory...
Checking Non_Fire directory...
Checking _Fire directory...
Cleaning process finished.


# Load the data

In [4]:
# Chemin vers le répertoire principal des données
data_dir = 'Data'
train_dir = os.path.join(data_dir, 'Train_Data')
test_dir = os.path.join(data_dir, 'Test_Data')

# Paramètres
img_height, img_width = 224, 224  # Taille des images (à adapter selon votre besoin)
batch_size = 32 #Taille des batchs
validation_split = 0.15 # Pourcentage de validation


In [5]:


train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',  # Infère les labels à partir des noms de dossiers
    label_mode='binary', #Labels binaires car 2 classes (Fire et Non_Fire)
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=123, #Important pour la reproductibilité
    validation_split=validation_split,
    subset='training'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=123,
    validation_split=validation_split,
    subset='validation'
)
# Chargement des données de test
test_dir = os.path.join(data_dir, 'Test_Data')
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 4611 files belonging to 2 classes.
Using 3920 files for training.
Found 4611 files belonging to 2 classes.
Using 691 files for validation.
Found 50 files belonging to 2 classes.


### Normalization de la data 

In [6]:
from tensorflow.keras.layers import Rescaling

normalization_layer = Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))


In [7]:
def create_model(neurons=32 , filter = 32, cnnlayers=2 ,annlayers=1) : 
    model = Sequential()
    
    model.add(Conv2D(filter , (3,3),activation='relu',input_shape = (img_height, img_width , 3)))
    model.add(MaxPooling2D(2,2))
    
    for _ in range (cnnlayers-2) : 
        model.add(Conv2D(filter , (3, 3) , activation='relu'))
        model.add(MaxPooling2D(2,2))
        model.add(Dropout(0.2))
        
    model.add(Conv2D(32 , (3,3),activation='relu'))
    model.add(MaxPooling2D(2,2))
    
    model.add(Flatten())
    model.add(Dense(32 , activation='relu'))
    
    for _ in range (annlayers-1) : 
        model.add(Dense(neurons , activation='relu'))
        model.add(Dropout(0.2))
        
    model.add(Dense(1 , activation='sigmoid'))
    
    model.compile(optimizers = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'] )
    
    
    return model 
        
        
    
    

In [8]:
from scikeras.wrappers import KerasClassifier


model = KerasClassifier(neurons=32 , filter = 32, cnnlayers=2 ,annlayers=1, build_fn = create_model , verbose =1)

In [9]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64],
    'filter' : [16, 32],
    'annlayers': [1, 2],
    'cnnlayers': [2, 3, 4],
    'epochs': [10]
}

In [10]:
import numpy as np

# Convert the tf.data.Dataset to NumPy arrays
def dataset_to_numpy(dataset):
    X, y = [], []
    for images, labels in dataset:
        X.append(images.numpy())
        y.append(labels.numpy())
    return np.concatenate(X), np.concatenate(y)

# Convert train_ds to NumPy arrays
X_train, y_train = dataset_to_numpy(train_ds)

In [12]:
from sklearn.model_selection import GridSearchCV

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 36 candidates, totalling 108 fits


MemoryError: Unable to allocate 1.47 GiB for an array with shape (2613, 224, 224, 3) and data type float32

In [13]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
